## Catch a Falling item


## Placeholder code: iiwa Simulation

As a starting point, the code below sets up a simple simulation with a single iiwa, visualized in Meshcat.

### Setup and imports
Let us first get our imports out of the way:

In [12]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import trimesh
from pydrake.all import (
    AddFrameTriadIllustration,
    BasicVector,
    Concatenate,
    Context,
    Diagram,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MultibodyPlant,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    Rgba,
    RigidTransform,
    RobotDiagram,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    Trajectory,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.icp import IterativeClosestPoint
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    LoadScenario,
    MakeHardwareStation,
    RobotDiagram,
)
from manipulation.utils import RenderDiagram

import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import trimesh
from pydrake.all import (
    AddFrameTriadIllustration,
    BasicVector,
    Concatenate,
    Context,
    Diagram,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MultibodyPlant,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    Rgba,
    RigidTransform,
    RobotDiagram,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    Trajectory,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.icp import IterativeClosestPoint
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    LoadScenario,
    MakeHardwareStation,
    RobotDiagram,
)
from manipulation.utils import RenderDiagram


# Next



import os
import time
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.trajectories.rrt_planner.robot import (
    ConfigurationSpace,
    Range,
)
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import (
    RRT,
    TreeNode,
)
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation

/usr/local/lib/python3.12/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [13]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [14]:
# Build the table_sdf
size_x = 0.8
size_y = 0.6
thickness = 0.05
mass = 10.0
mu_static = 1.0
mu_dynamic = 0.8
dissipation = 1.0

Ixx = (1.0/12.0) * mass * (size_y**2 + thickness**2)
Iyy = (1.0/12.0) * mass * (size_x**2 + thickness**2)
Izz = (1.0/12.0) * mass * (size_x**2 + size_y**2)

table_sdf = f"""<?xml version="1.0"?>
<sdf version="1.7" xmlns:drake="http://drake.mit.edu">
  <model name="table">
    <link name="table_top">
      <inertial>
        <mass>{mass}</mass>
        <inertia>
          <ixx>{Ixx}</ixx><iyy>{Iyy}</iyy><izz>{Izz}</izz>
          <ixy>0</ixy><ixz>0</ixz><iyz>0</iyz>
        </inertia>
      </inertial>

      <visual name="visual">
        <geometry><box><size>{size_x} {size_y} {thickness}</size></box></geometry>
        <material>
          <ambient>0.6 0.6 0.6 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
        </material>
      </visual>

      <collision name="collision">
        <geometry><box><size>{size_x} {size_y} {thickness}</size></box></geometry>
        <drake:proximity_properties>
          <drake:hunt_crossley_dissipation>{dissipation}</drake:hunt_crossley_dissipation>
          <drake:mu_static>{mu_static}</drake:mu_static>
          <drake:mu_dynamic>{mu_dynamic}</drake:mu_dynamic>
        </drake:proximity_properties>
      </collision>
    </link>
  </model>
</sdf>
"""

assets_dir = Path("assets")
assets_dir.mkdir(parents=True, exist_ok=True)
(assets_dir / "table.sdf").write_text(table_sdf)
print("Wrote", (assets_dir / "table.sdf").resolve())

Wrote /workspaces/Projects/robotic_manipulation/00-Pset-9/assets/table.sdf


In [15]:
# The cracker box SDF file is already in the assets folder

In [16]:
# Add the directives for the IIWA arm, table, and cracker box


def generate_bimanual_IIWA14_with_assets_directives_file() -> (
    tuple[Diagram, RobotDiagram]
):
    table_sdf = f"{Path.cwd()}/assets/table.sdf"
    cracker_box_sdf = f"{Path.cwd()}/assets/003_cracker_box.sdf"

    directives_yaml = f"""directives:
- add_model:
    name: table
    file: file://{table_sdf}
- add_weld:
    parent: world
    child: table::table_top
    X_PC:
        translation: [0.0, 0.0, -.025]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: table::table_top
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0.3, 0.6, 0]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: cracker_box
    file: file://{cracker_box_sdf}
"""
    os.makedirs("directives", exist_ok=True)

    with open(
        "directives/bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml", "w"
    ) as f:
        f.write(directives_yaml)


generate_bimanual_IIWA14_with_assets_directives_file()

In [17]:
def create_camera_directives() -> None:
    camera_directives_yaml = """
directives:
- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-120.0, 0.0, 180.0]}
        translation: [0, 0.8, 0.5]

- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera0_origin
    child: camera0::base

- add_frame:
    name: camera1_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-125, 0.0, 90.0]}
        translation: [0.8, 0.1, 0.5]

- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera1_origin
    child: camera1::base

- add_frame:
    name: camera2_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-120.0, 0.0, -90.0]}
        translation: [-0.8, 0.1, 0.5]

- add_model:
    name: camera2
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera2_origin
    child: camera2::base
"""
    with open("directives/camera_directives.dmd.yaml", "w") as f:
        f.write(camera_directives_yaml)


create_camera_directives()

In [18]:
def create_bimanual_IIWA14_with_assets_and_cameras_scenario() -> None:
    # Build absolute file:// URIs for the two directives files
    dmd1 = (Path.cwd() / "directives" / "bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml").resolve().as_posix()
    dmd2 = (Path.cwd() / "directives" / "camera_directives.dmd.yaml").resolve().as_posix()

    # Part 1: directives (use file:// URIs)
    scenario_yaml = f"""# Station scenario
directives:
  - add_directives:
      file: file://{dmd1}
  - add_directives:
      file: file://{dmd2}
"""

    # Part 2: cameras (DICT form; fields your schema accepts)
    scenario_yaml += """
cameras:
  camera0:
    name: camera0
    depth: true
    X_PB:
      base_frame: camera0::base
  camera1:
    name: camera1
    depth: true
    X_PB:
      base_frame: camera1::base
  camera2:
    name: camera2
    depth: true
    X_PB:
      base_frame: camera2::base
"""

    # Part 3: drivers
    scenario_yaml += """
model_drivers:
  iiwa: !IiwaDriver
    control_mode: position_only
    hand_model_name: wsg
  wsg: !SchunkWsgDriver {}
"""

    os.makedirs("scenarios", exist_ok=True)

    with open(
        "scenarios/bimanual_IIWA14_with_table_and_initials_and_assets_and_cameras.scenario.yaml",
        "w",
    ) as f:
        f.write(scenario_yaml)


scenario_yaml = create_bimanual_IIWA14_with_assets_and_cameras_scenario()

In [19]:
scenario_file = "scenarios/bimanual_IIWA14_with_table_and_initials_and_assets_and_cameras.scenario.yaml"


def create_bimanual_IIWA14_with_table_and_initials_and_assets() -> (
    tuple[DiagramBuilder, RobotDiagram]
):
    # TODO: Load the scenario created above into a Scenario object
    scenario = LoadScenario(
        filename="scenarios/bimanual_IIWA14_with_table_and_initials_and_assets_and_cameras.scenario.yaml"
    )

    # TODO: Create HardwareStation with the scenario and meshcat
    station = MakeHardwareStation(scenario, meshcat=meshcat)

    # TODO: Make a DiagramBuilder, add the station, and build the diagram
    builder = DiagramBuilder()
    builder.AddSystem(station)

    # TODO: Return the builder AND the station (notice that here we will need both)
    return builder, station


In [20]:
# Function to compute the time to catch a falling object
def solve_catch_time(q0, v0, catch_height, buffer_time=0.5):
    # Buffer time for the object to be there beforehand
    # Simple physics calculation assuming constant acceleration due to gravity
    g = 9.81  # m/s^2
    delta_h = q0 - catch_height

    # Using the equation: v^2 = u^2 + 2*a*s
    # where v = final velocity (0 m/s at catch height), u = initial velocity (0 m/s), a = g, s = delta_h
    time_to_fall = np.sqrt((2 * delta_h) / g)


    total_time = time_to_fall + buffer_time
    return total_time

In [21]:
# Define the bot's arm trajectory
def make_bot_trajectory(
    X_Gs: list[RigidTransform],
    finger_values: np.ndarray,
    sample_times: list[float]
) -> tuple:
    """
    Create trajectories for gripper pose and finger positions.

    Returns:
        robot_velocity_trajectory: Spatial velocity trajectory for controller
        traj_wsg_command: Finger position trajectory
    """
    from pydrake.all import PiecewisePose, PiecewisePolynomial

    robot_position_trajectory = PiecewisePose.MakeLinear(sample_times, X_Gs)
    robot_velocity_trajectory = robot_position_trajectory.MakeDerivative()
    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(sample_times, finger_values)

    return robot_velocity_trajectory, traj_wsg_command

In [22]:
# Psu
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector):
        """
        fill in our code below.
        """
        # evaluate the V_G_port and q_port on the current context to get those values.
        V_G = V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)

        # update the positions of the internal _plant_context according to `q`.
        # HINT: you can write to a plant context by calling `self._plant.SetPositions`

        self._plant.SetPositions(self._plant_context, self._iiwa, q)

        # Compute the gripper jacobian
        # HINT: the jacobian is 6 x N, with N being the number of DOFs.
        # We only want the 6 x 7 submatrix corresponding to the IIWA
        J_full = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kV,
            self._G, np.zeros(3),   # p_BoQ_B = [0,0,0] at frame G origin
            self._W,                # measured in world (Ao = W)
            self._W                 # expressed in world (E = W)
        )
        J = J_full[:, self.iiwa_start : self.iiwa_end + 1]

        # compute `v` by mapping the gripper velocity (from the V_G_port) to the joint space
        lam2 = 1e-6
        JJt = J @ J.T
        J_pinv = J.T @ np.linalg.inv(JJt + lam2 * np.eye(6))
        v = J_pinv @ V_G  # 7x1 joint velocity command

        output.SetFromVector(v)

In [38]:
# Create a specific trajectory for what we want

p0_letter = np.array([0.0, 0.0, 8])
R_WL = RollPitchYaw(np.pi/2, np.pi, np.pi/2)

opened = 0.107
closed = 0.0
min_spacing = 0.15



# Calclulate the trajectory for the book getting to a specific height
catch_height = 0.3
v0_letter = np.array([0.0, 0.0, 0.0])

t_catch = solve_catch_time(p0_letter[2], v0_letter[2], catch_height, buffer_time = 0.15)
if t_catch is None:
    print("ERROR: Can't calculate catch time!")
    t_catch = 1.0
print(f"Calculated catch time: {t_catch:.2f} seconds")


# Define key poses for the gripper
R_WG_down = RollPitchYaw(0, 0, 0).ToRotationMatrix()


# Hovering high, a bit back from the box
p_WG_initial = np.array([0.5, 0.3, 0.5])
p_WG_initial = np.array([0.0, 0.0, 0.5])

# Move toward the box, still safely above
p_WG_prepick = np.array([0, 0, 0.5])

# p_WG_prepick = np.array([0, 0, .3])


# Directly above the box, close to catch/grasp height
# p_WG_pick = np.array([0.0, -0.1, 0.5])
p_WG_pick = np.array([0, 0, .5])


X_WGinitial = RigidTransform(R_WG_down, p_WG_initial)

X_WGprepick = RigidTransform(R_WG_down, p_WG_prepick)
X_WGpick    = RigidTransform(R_WG_down, p_WG_pick)

# X_WGprepick = X_WGinitial
# X_WGpick    = X_WGinitial


if t_catch < 2.0:
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, max(1.2, t_catch * 0.3)), #Change max back to 0.5 from 2
        ("X_WGpick", X_WGpick, opened, max(1.2, t_catch * 0.7)), #change max back to 0.8 from 2
        ("X_WGpick", X_WGpick, closed, t_catch),
        # ("X_WGpick", X_WGpick, closed, t_catch + 1.5),
    ]
else:
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, t_catch - 1.5),
        ("X_WGpick", X_WGpick, opened, t_catch - 0.3),
        ("X_WGpick", X_WGpick, closed, t_catch),
        # ("X_WGpick", X_WGpick, closed, t_catch + 1.5),
    ]



sample_times = [keyframe[3] for keyframe in keyframes]
for i in range(1, len(sample_times)):
    if sample_times[i] - sample_times[i-1] < min_spacing:
        sample_times[i] = sample_times[i-1] + min_spacing

gripper_poses = [keyframe[1] for keyframe in keyframes]
finger_states = np.asarray([keyframe[2] for keyframe in keyframes]).reshape(1, -1)
traj_V_G, traj_wsg_command = make_bot_trajectory(gripper_poses, finger_states, sample_times)

Calculated catch time: 1.40 seconds


In [39]:
# Set up the simulator.
builder, station = create_bimanual_IIWA14_with_table_and_initials_and_assets()
plant = station.GetSubsystemByName("plant")







# TODO : Add Trajectory System and controls
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
controller = builder.AddSystem(PseudoInverseController(plant))
integrator = builder.AddSystem(Integrator(7))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connct everything
builder.Connect(V_G_source.get_output_port(), controller.GetInputPort("V_WG"))
builder.Connect(controller.GetOutputPort("iiwa.velocity"), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))

# visualize_everything(station, plant)

# Build the diagram
# visualize_everything(station, plant)
diagram = builder.Build()

# Create simulator and context
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)

In [40]:
def debug_gripper_pose(plant: MultibodyPlant, q: np.ndarray, label: str = ""):
    """Print world->gripper transform for a given joint configuration q."""
    context = plant.CreateDefaultContext()
    iiwa = plant.GetModelInstanceByName("iiwa")
    plant.SetPositions(context, iiwa, q)

    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")

    X_WG = plant.CalcRelativeTransform(context, world_frame, gripper_frame)

    print(f"\n=== Gripper pose ({label}) ===")
    print("q =", q)
    print("translation (p_WG):", X_WG.translation())
    print("rotation R_WG:\n", X_WG.rotation().matrix())


In [ ]:
# Inverse Kinematics


def solve_ik_for_pose(
    plant: MultibodyPlant,
    X_WG_target: RigidTransform,
    q_nominal: tuple = tuple(
        np.array([0.0, 0.1, 0.0, -1.2, 0.0, 1.6, 0.0])  # the inital joint poisitions
    ),
    theta_bound: float = 0.01 * np.pi,
    pos_tol: float = 0.015,
) -> tuple:
    """
    Solve IK for a single end-effector pose.

    Args:
        plant: A MultibodyPlant with the iiwa + gripper model.
        X_WG_target: Desired gripper pose in world frame.
        q_nominal: Nominal joint angles for joint-centering.
        theta_bound: Orientation tolerance (radians).
        pos_tol: Position tolerance (meters).

    Returns:
        q_solution: 7 element tuple representing the Optimal
        joint configuration. Each element of the tuple is a float.
    """
    print("\n----------------------------------------")
    print(f"Solving IK for target position: {X_WG_target.translation()}")
    print("Target rotation R_WG:\n", X_WG_target.rotation().matrix())

    world_frame = plant.world_frame()

    gripper_frame = plant.GetFrameByName("body")

    debug_gripper_pose(plant, np.array(q_nominal), label="q_nominal (before IK)")




    ik = InverseKinematics(plant)
    q_vars = ik.q()[:7]
    prog = ik.prog()

    # TODO: Add Orientation constraint
    R_WG_des = X_WG_target.rotation()
    ik.AddOrientationConstraint(
        frameAbar=world_frame,
        R_AbarA=R_WG_des,           # desired world->gripper rotation
        frameBbar=gripper_frame,    # <-- use frameBbar (not frameB)
        R_BbarB=RotationMatrix(),   # identity in the gripper local frame
        theta_bound=theta_bound,
    )
    # TODO: Add Position constraint
    p_WG_des = X_WG_target.translation()
    p_min = p_WG_des - np.array([pos_tol, pos_tol, pos_tol])
    p_max = p_WG_des + np.array([pos_tol, pos_tol, pos_tol])
    ik.AddPositionConstraint(
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        frameA=world_frame,
        p_AQ_lower=p_min,
        p_AQ_upper=p_max,
    )
    # TODO: Add Joint centering cost
    error = q_vars - q_nominal
    prog.AddQuadraticCost(error.dot(error))

    # Initial guess
    prog.SetInitialGuess(q_vars, q_nominal)

    result = Solve(prog)
    if not result.is_success():
        raise RuntimeError("IK did not succeed")

    return tuple(result.GetSolution(q_vars))


# X_WGinitial = RigidTransform(R_WG_down, p_WG_initial)
# X_WGprepick = RigidTransform(R_WG_down, p_WG_prepick)
# X_WGprepick = RigidTransform(R_WG_down, p_WG_initial)
# X_WGprepick = X_WGinitial
# X_WGpick    = RigidTransform(R_WG_down, p_WG_pick)


q_initial  = solve_ik_for_pose(plant, X_WGinitial)
q_approach = solve_ik_for_pose(plant, X_WGprepick)
# q_goal = solve_ik_for_pose(plant, X_WGpick, q_nominal=q_approach)
q_goal = solve_ik_for_pose(plant, X_WGprepick, q_nominal=q_approach)



----------------------------------------
Solving IK for target position: [0.  0.  0.5]
Target rotation R_WG:
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]

=== Gripper pose (q_nominal (before IK)) ===
q = [ 0.   0.1  0.  -1.2  0.   1.6  0. ]
translation (p_WG): [ 0.13438304 -0.3         0.65432158]
rotation R_WG:
 [[ 3.04373830e-16 -2.39249329e-01  9.70958165e-01]
 [-1.00000000e+00 -3.40953638e-16  2.92549835e-16]
 [ 1.86849850e-16 -9.70958165e-01 -2.39249329e-01]]

----------------------------------------
Solving IK for target position: [0.  0.  0.5]
Target rotation R_WG:
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]

=== Gripper pose (q_nominal (before IK)) ===
q = [ 0.   0.1  0.  -1.2  0.   1.6  0. ]
translation (p_WG): [ 0.13438304 -0.3         0.65432158]
rotation R_WG:
 [[ 3.04373830e-16 -2.39249329e-01  9.70958165e-01]
 [-1.00000000e+00 -3.40953638e-16  2.92549835e-16]
 [ 1.86849850e-16 -9.70958165e-01 -2.39249329e-01]]

----------------------------------------
Solving IK for

In [42]:
def get_model_body(plant: MultibodyPlant):
    ""

    # Get the letter body
    # letter_model = plant.GetModelInstanceByName(f"{your_initial}_letter")
    # letter_body = plant.GetBodyByName(f"{your_initial}_body_link", letter_model)


    # - add_model:
    #     name: cheez_it
    #     file: package://manipulation/hydro/003_cracker_box.sdf

    letter_model = plant.GetModelInstanceByName(f"cracker_box")
    letter_body = plant.GetBodyByName("base_link_cracker", letter_model)

    return letter_model, letter_body




# Modify the position of the book (Ideally could move position stuff down here)
letter_model, letter_body = get_model_body(plant)
plant.SetFreeBodyPose(
    plant_context,
    letter_body,
    RigidTransform(R_WL, p0_letter),
)


# Set integrator to match actual starting position
# integrator.set_integral_value(
#     integrator.GetMyContextFromRoot(context),
#     plant.GetPositions(
#         plant.GetMyContextFromRoot(context),
#         plant.GetModelInstanceByName("iiwa"),
#     ),
# )

integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    np.array(q_initial),
)



# integrator.set_integral_value(
#     integrator.GetMyContextFromRoot(context),
#     plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa")),
# )



# Get the station context to fix input ports
station_context = station.GetMyContextFromRoot(context)

# Fix the required input ports with default values
# Get the initial joint positions from the plant (from default_joint_positions in directives)
# q0 = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0])  # Default positions from directives
# station.GetInputPort("iiwa.position").FixValue(station_context, q0)
# station.GetInputPort("wsg.position").FixValue(station_context, [0.1])  # Open gripper

# THIS IS THE KEY LINE - it publishes the scene to meshcat
diagram.ForcedPublish(context)
print(f"Simulation will run for {traj_V_G.end_time()} seconds")



meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5)
meshcat.StopRecording()
meshcat.PublishRecording()

Simulation will run for 1.4999999999999998 seconds
